In [41]:
from glob import glob
import os
from os.path import join
from numpy import genfromtxt
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import csv
%matplotlib inline

In [83]:
DET_METRIC = 'any'
def get_dr(sdir,clf_d):
        fn = join(sdir,clf_d,'recall','fold_test{}.csv'.format(DET_METRIC))
        df = pd.read_csv(fn)
        dr = df.loc[df['Label']=='Macro Average of Attacks','Rc'].values[0]
        return dr


def get_sampled_ben_dr(sdir,clf_d):
        fn = join(sdir,clf_d,'eval','fold_test{}.csv'.format(DET_METRIC))
        df = pd.read_csv(fn)
        det_rate = df.loc[df['Label']=='Benign','Rc'].values[0]
        return det_rate

def get_benign_obsr_rate(sdir):
        fn = join(sdir,'observation_rate.csv')
        df = pd.read_csv(fn)
        ben_obsr_rate = df.loc[df['Label']=='Benign','Observation rate'].values[0]     
        return ben_obsr_rate
        
def get_far(sdir,clf_d):
    sampled_benign_det_rate = get_sampled_ben_dr(sdir,clf_d)
    sampled_far = 100.-sampled_benign_det_rate
    ben_obsr_rate = get_benign_obsr_rate(sdir)
    far = sampled_far* ben_obsr_rate/100.
    return round(far,2)

In [84]:
def get_sampler_name(sdir):
    dir_name  = os.path.basename(sdir)
    sampler_name = dir_name[:dir_name.find('_')]
    return sampler_name


def write_dr(dataroot,dr_results):
    with open (join(dataroot,'dr_{}.csv'.format(DET_METRIC)),'w') as csvfile:
        w = csv.writer(csvfile)
        w.writerow(['Sampling technique','DT','RF','CNN'])

        if DET_METRIC=='any':
            w.writerow(['NS',99.09, 99.09, 96.74 ])
        elif DET_METRIC == 'all':
            w.writerow(['NS',81.95, 89.94, 96.22])
            
        for key in sorted(dr_results.keys()):
            w.writerow([key]+dr_results[key])
        

def write_far(dataroot,far_results):
    with open (join(dataroot,'far_{}.csv'.format(DET_METRIC)),'w') as csvfile:
        w = csv.writer(csvfile)
        w.writerow(['Sampling technique','DT','RF','CNN'])

        if DET_METRIC=='any':
            w.writerow(['NS', 0.12, 0,  0.05])
        elif DET_METRIC == 'all':
            w.writerow(['NS', 2.44, 0.34, 2.89])

        for key in sorted(far_results.keys()):
            w.writerow([key]+far_results[key])

In [90]:
dataroot = '/data/juma/data/ids18/CSVs_r_1.0_m_1.0/SR_0.1/'
clf_dirs = ['c_tree_b_explicit_mf_auto_msl_3_md_25',
            'c_forest_b_explicit_n_100_bootstrap_True_mf_auto_msl_3_ms_0.01_md_25',
            'c_cnn_optim_Adam_lr_0.001_reg_0_bs_1024_b_explicit']

dr_results = {}
far_results = {}
for sdir in sorted(glob(join(dataroot,'*_l'))):
    dr_list = []
    far_list = []
    for clf_d in clf_dirs:
        dr_list.append(get_dr(sdir,clf_d))
        far_list.append(get_far(sdir,clf_d))
        
    dr_results[get_sampler_name(sdir)] = dr_list
    far_results[get_sampler_name(sdir)] = far_list
    
write_dr(dataroot,dr_results)
write_far(dataroot,far_results)